# First model 

This model will try to predict wether pe is present or not on a single image (grey level)

## Imports

In [1]:
import numpy as np
from balance import *

In [15]:
import torch
torch.cuda.empty_cache()

print(f'PyTorch version: {torch.__version__}')
print("GPU found :)" if torch.cuda.is_available() else "No GPU :(")

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

PyTorch version: 1.7.1
GPU found :)


# Load datasets

In [3]:
channel0_dataset = torch.load('datasets/channel0.pt')
channel1_dataset = torch.load('datasets/channel1.pt')
channel2_dataset = torch.load('datasets/channel2.pt')

## Simple CNN from scratch

In [4]:
class ModelV1(nn.Module):
    
    def __init__(self, input_channels):
        """Simple Convnet with 2 convolution layer + pooling, with 2 fully connected at the end"""
        super().__init__()
        
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3)
        self.pool = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(16, 32, 3)
        self.fc1 = nn.Linear(32*62*62 , 120)
        self.fc2 = nn.Linear(120, 1)
        self.sigmoid = nn.Sigmoid()
        
        
    def forward(self, x):
        
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32*62*62) # Flatten
        x = F.relu(self.fc1(x))
        
        # Binary classification
        out = self.sigmoid(self.fc2(x))
        
        return out

## Split data into train and test

In [7]:
dataset = channel0_dataset
batch_size = 24

train_loader, test_loader = balance_train_test_split(dataset, ratio=0.2, batch_size=batch_size)

Train size: 5016
Test size: 1254


## Model Training

In [20]:
num_epoch = 10
step_count = len(train_loader)
loss_function = nn.BCELoss()
model = ModelV1(input_channels=1).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(num_epoch):
    for i, sample in enumerate(train_loader):
        
        image = sample['image'].to(device, dtype=torch.float)
        target = sample['target'].to(device, dtype=torch.float)
        target = target.view(target.shape[0], 1)
        
        # Reset gradiant
        optimizer.zero_grad()
        
        # Forward pass
        pred = model(image)
        
        # Compute loss
        loss = loss_function(pred, target)
        
        # Backprop
        loss.backward()
        optimizer.step()
        
        # Debug
        if((i+1) % step_count/10 == 0):
            print(
                        f"Epoch [{epoch + 1}/{num_epoch}]"
                        f", step [{i + 1}/{step_count}]"
                        f", loss: {loss.item():.4f}"
                    )

RuntimeError: CUDA out of memory. Tried to allocate 58.00 MiB (GPU 0; 1.96 GiB total capacity; 1.23 GiB already allocated; 10.00 MiB free; 1.28 GiB reserved in total by PyTorch)

In [ ]:
# Save model 
torch.save(model.state_dict(), './models/model_v1.h5')

## Test the model

In [13]:
# Load model for evaluation
trained_model = ModelV1(input_channels=1)
trained_model.load_state_dict(torch.load('./models/model_v1.h5'))
trained_model.to(device)

RuntimeError: CUDA out of memory. Tried to allocate 58.00 MiB (GPU 0; 1.96 GiB total capacity; 1.23 GiB already allocated; 28.50 MiB free; 1.28 GiB reserved in total by PyTorch)

In [ ]:
correct = 0
with torch.no_grad():
    for sample in test_loader:
        
        image = sample['image'].to(device, dtype=torch.float)
        target = sample['target'].to(device)
        
        output = model(image)
        prob, index = torch.max(output, 1)
        correct += (index == target).sum().item()

total = len(test_loader)
print(f'Accuracy of the network on the {total} test images: {100 * correct / total}')